https://www.gdal.org/gdal_retile.html



## Import

In [1]:
import glob
import os
import shutil  # Kopiering av filer

import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from osgeo import gdal, ogr  # GDAL

import gdal_merge

# Python kode som delar opp bildet.
import gdal_retile
import geopandas  # http://geopandas.org/reference.html

In [2]:
# Check that unpacked files are there. Unpack them?
!ls ../Raw_data

20180705-biri_02_SWIR_384me_SN3126_raw_rad_bsq_float32_geo.dat
20180705-biri_02_SWIR_384me_SN3126_raw_rad_bsq_float32_geo.dat.aux.xml
20180705-biri_02_SWIR_384me_SN3126_raw_rad_bsq_float32_geo.hdr
20180705-biri_02_SWIR_384me_SN3126_raw_rad_bsq_float32_geo.zip
20180705-biri_02_VNIR_1800_SN00827_raw_rad_bsq_float32_geo.dat
20180705-biri_02_VNIR_1800_SN00827_raw_rad_bsq_float32_geo.dat.aux.xml
20180705-biri_02_VNIR_1800_SN00827_raw_rad_bsq_float32_geo.hdr
20180705-biri_02_VNIR_1800_SN00827_raw_rad_bsq_float32_geo.zip


In [3]:
# Global variables:
imagestripes_names = glob.glob("../Raw_data/*.dat")  # Lists every imagestripe
imagestripes_out_dir = "../Merged_image_stripes/"


name_imagefiles = glob.glob("../Merged_image_stripes/*.tif")  # Lists every imagefile
# ame_shapefile = "shapefile/utsnit.shp"
name_csv = "tile_envelope_coordinates.csv"
in_directory = "../Sorted_tiles"
out_directory = "../Sorted_tiles/mis_tiles"

# Size of output tile
tile_with = "50"
tile_heigth = "50"

## Merge bands

In [ ]:
imagestripes_names

In [ ]:
%time
# Run merge function for each set of VNIR and SWNIR stripes
for i, k in enumerate(imagestripes_names[::2]):
    print(f'Prossesing stripe {i+1} of {len(imagestripes_names)}')
    print(f'Merging {imagestripes_names[1]} and {imagestripes_names[0]}')

    
    # The highes resolution image as fist input. In this case VNIR before Swnir.
    cmd = 'gdal_merge.py -o '+ imagestripes_out_dir +'stripe_merged_'+ str(2+i) +'.tif -separate -v -ot Float32 -of GTiff '+ imagestripes_names[i+1] +' '+ imagestripes_names[i]
    
    os.system(cmd)

In [ ]:
cmd

## Make tiles

In [ ]:
name_imagefiles = glob.glob("../Merged_image_stripes/*.tif")  # Lists every imagefile

In [ ]:
%time
# Run retilefunction for each image
for i, image_name in enumerate(name_imagefiles):
    print(f'Prossesing image {i+1} of {len(name_tiles)}')
    
    cmd = 'gdal_retile.py -ps ' + tile_with +' '+ tile_heigth + ' -overlap 0 -levels 1 -r near -ot Float32 -csv '+ str(i) + name_csv + ' -csvDelim "," -targetDir '+'\''+in_directory+'\''+ ' ' +'\''+image_name+'\''
    #print(cmd)
    os.system(cmd)

### Delete empyt tiles or tiles with irregular shape 

In [ ]:
list_tiles = glob.glob("../Sorted_tiles/*.tif")
tiles_to_be_removed = []

In [ ]:
def get_input(path):

    img = imageio.imread(path)

    return img

In [ ]:
# Checks tiles and delete the irregular ones.
for tile_path in list_tiles:
    image = get_input(tile_path)

    if not image.any():  # Checks if image is only zeroes, no data.
        tiles_to_be_removed.append(os.path.basename(tile_path))
        # os.remove(tile_path)

    elif image.shape != (
        int(tile_with),
        int(tile_heigth),
        474,
    ):  # Checks if image is correctly shaped
        tiles_to_be_removed.append(os.path.basename(tile_path))
        # os.remove(tile_path)

In [ ]:
print(f"{len(tiles_to_be_removed)} tiles was deleted.")

## Find categories
Label 99 means that the tile contains no MIS-label.

In [ ]:
MIS_code_key = pd.read_excel(
    io="../Prosjekt_Honne_data_Landbruksdirektoratet/Koder Mis-nin i shape pilot 2.0.xlsx",
    sheet_name="LM.kode",
)
MIS_code_key

In [ ]:
# Read mis-polygons
mis_geometry_mjosen = geopandas.read_file(
    "../Prosjekt_Honne_data_Landbruksdirektoratet/Leveranse_MiS_201017_Mjosen/MiS_NiN_Biri_Mjosen_2017.shp"
)

mis_geometry_mjosen.head()

In [ ]:
# Plot mis-polygons
plt.figure()
mis_geometry_mjosen.plot()
plt.show()

### Test polygon

In [ ]:
mis_geometry_test = geopandas.read_file(
    "../Qgis/test_polygon/testpolygon_rett_epsg.shp"
)
mis_geometry_test.head()

In [ ]:
# Plot mis-polygons
plt.figure()
mis_geometry_test.plot()
plt.show()

## Sort tiles

In [ ]:
# Merge all csv files to one big
all_files = glob.glob(
    os.path.join(in_directory, "*.csv")
)  # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (
    pd.read_csv(f, names=["Tile_name", "X_ul", "X_lr", "Y_ul", "Y_lr"])
    for f in all_files
)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
concatenated_df.count()

In [ ]:
concatenated_df.head()

In [ ]:
# Remove tiles found to be irregular or not wanted
concatenated_df = concatenated_df[
    ~concatenated_df["Tile_name"].isin(tiles_to_be_removed)
]
concatenated_df.reset_index(drop=True, inplace=True)
concatenated_df.head()

In [ ]:
concatenated_df.count()

In [ ]:
# Create a geometry of every tile (bounding box)
# https://gis.stackexchange.com/questions/285336/convert-polygon-bounding-box-to-geodataframe
b = [
    geopandas.base.box(l, b, r, t)
    for l, b, r, t in zip(
        concatenated_df.X_ul,
        concatenated_df.Y_lr,
        concatenated_df.X_lr,
        concatenated_df.Y_ul,
    )
]

gdf = geopandas.GeoDataFrame(concatenated_df, geometry=b)

# Create dataframe to save labels
labeled_tiles = pd.DataFrame(gdf["Tile_name"])
labeled_tiles.index.name = "ID"

# Create label collum, label 99 means no "no MIS label"
labeled_tiles["LIVSM1"] = int(99)
labeled_tiles["LIVSM2"] = int(99)

In [ ]:
labeled_tiles.head()

In [ ]:
# Modifisert
# Bruk informasjonen i mis_geometry.LIVSM_XXX for å tilegne klasse i sorteringa. Lagre dette i csv-fil eller inn i
# concatenated_df. Då kan ein lese navn på flisa og i same slengen også leggje til miskoden inn i generatoren. Altså
# både x (imageread(filnamn)) og y (klassekoden).
# Check if a tile is inside a mis-polygon. Sorts tiles.
for mis_geom_num, mis_geom_row in mis_geometry_mjosen.iterrows():
    print(f"Checking polygon {mis_geom_num + 1} of {len(mis_geometry_mjosen)}")

    # Check if tile geometries is inside a mis-polygon
    for index, row in gdf.iterrows():
        # If tile is inside, sort it into folder
        if mis_geom_row["geometry"].contains(row["geometry"]):
            # shutil.move(in_directory + '/' + row['Tile_name'], out_directory + '/' + row['Tile_name'])
            labeled_tiles.loc[labeled_tiles.index[index], "LIVSM1"] = int(
                mis_geom_row["LIVSM1"]
            )
            labeled_tiles.loc[labeled_tiles.index[index], "LIVSM2"] = int(
                mis_geom_row["LIVSM2"]
            )

            # print(f'Label {gdf["mis_label"][index]} given tile {row["Tile_name"]}')
            print(
                "Label LIVSM1 {0} LIVSM2 {1} given tile {2}".format(
                    labeled_tiles["LIVSM1"][index],
                    labeled_tiles["LIVSM2"][index],
                    row["Tile_name"],
                )
            )

### Test polygon

In [ ]:
# Modifisert
# Bruk informasjonen i mis_geometry.LIVSM_XXX for å tilegne klasse i sorteringa. Lagre dette i csv-fil eller inn i
# concatenated_df. Då kan ein lese navn på flisa og i same slengen også leggje til miskoden inn i generatoren. Altså
# både x (imageread(filnamn)) og y (klassekoden).
# Check if a tile is inside a mis-polygon. Sorts tiles.
for mis_geom_num, mis_geom_row in mis_geometry_test.iterrows():
    print(f"Checking polygon {mis_geom_num + 1} of {len(mis_geometry_test)}")

    # Check if tile geometries is inside a mis-polygon
    for index, row in gdf.iterrows():
        # If tile is inside, sort it into folder
        if mis_geom_row["geometry"].contains(row["geometry"]):
            # shutil.move(in_directory + '/' + row['Tile_name'], out_directory + '/' + row['Tile_name'])
            labeled_tiles.loc[labeled_tiles.index[index], "LIVSM1"] = int(
                mis_geom_row["LIVSM1"]
            )
            labeled_tiles.loc[labeled_tiles.index[index], "LIVSM2"] = int(
                mis_geom_row["LIVSM2"]
            )

            # print(f'Label {gdf["mis_label"][index]} given tile {row["Tile_name"]}')
            print(
                "Label LIVSM1 {0} LIVSM2 {1} given tile {2}".format(
                    labeled_tiles["LIVSM1"][index],
                    labeled_tiles["LIVSM2"][index],
                    row["Tile_name"],
                )
            )

MultiLabelBinarizer is used to make a multilabel binary label. This labelvector is used by the model.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mergedlabels = [
    list(pair)
    for pair in zip(labeled_tiles["LIVSM1"].values, labeled_tiles["LIVSM2"].values)
]

# If the tile only have one label, only one label is given. 0 is ignored == no label.
mlb = MultiLabelBinarizer(classes=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 99])
binarized_labels = mlb.fit_transform(mergedlabels)

In [ ]:
labels_as_colums = pd.DataFrame(binarized_labels, columns=mlb.classes)
labels_as_colums.head()

In [ ]:
# Convert binarized_labels to dataframe
bin_labels_dict = {
    "index": list(range(len(binarized_labels))),
    "mult_bin_labels": binarized_labels.tolist(),
}
binarized_labels_dataframe = pd.DataFrame(
    bin_labels_dict, columns=["index", "mult_bin_labels"]
)

# view the dataset
binarized_labels_dataframe.head()

In [ ]:
labeled_tiles_bin = pd.concat(
    [labeled_tiles, binarized_labels_dataframe["mult_bin_labels"], labels_as_colums],
    axis=1,
)
labeled_tiles_bin.tail()

In [ ]:
# Save csv to file
labeled_tiles_bin.to_csv("../Qgis/labeled_tiles.csv")

labeled_tiles_bin.head()

In [ ]:
labeled_tiles_bin[5].value_counts()

## Read images

In [ ]:
import random

import imageio
import scipy.ndimage

im = imageio.imread("../Sorted_tiles/stripe_merged_2_03_30.tif")

# no = random.randrange(-180,180,30)
# flipped_patch = scipy.ndimage.interpolation.rotate(im, no,axes=(1, 0), reshape=False, output=None, order=3, mode='mirror', cval=0.0, prefilter=False)

In [ ]:
flipped_patch.shape

In [ ]:
flipped_patch[0][0]

In [ ]:
im.shape

In [ ]:
kk = im.flatten()
kk.shape

In [ ]:
kk = np.atleast_3d(im.flatten())
kk.shape

In [ ]:
kks = im.reshape(632, 625, 3)
kks.shape

In [ ]:
kk = np.reshape(im, (625, 632, 3))
kk.shape

In [ ]:
395000

In [ ]:
for i in reversed(range(200, 900)):
    for j in range(200, 900):
        if i * j == 155000:
            print(i)
            print(j)

In [ ]:
im = imageio.imread("../Sorted_tiles/stripe_merged_2_04_30.tif")

standard_image = im

pca_standard = PCA(30)
pca_reshape = im.reshape(-1, 474)
standard_pca_image = pca_standard.fit_transform(pca_reshape)
standard_pca_image = standard_pca_image.reshape(50, 50, 30)
print(pca_standard.explained_variance_ratio_)


# Equalization
img_eq = exposure.equalize_hist(im)

pca_eq = PCA(30)
pca_reshape = img_eq.reshape(-1, 474)
eq_pca = pca_eq.fit_transform(pca_reshape)
eq_pca = eq_pca.reshape(50, 50, 30)

print(pca_eq.explained_variance_ratio_)


# Contrast stretching
p2, p98 = np.percentile(im, (2, 98))
img_rescale = exposure.rescale_intensity(im, in_range=(p2, p98))
# plt.imshow(img_rescale[:,:,0])

pca_stretch = PCA(30)
pca_reshape = img_rescale.reshape(-1, 474)
stretch_pca = pca_stretch.fit_transform(pca_reshape)
stretch_pca = stretch_pca.reshape(50, 50, 30)

print(pca_stretch.explained_variance_ratio_)

In [ ]:
plt.subplot(321)
plt.imshow(standard_image[:, :, 0])

plt.subplot(322)
plt.imshow(standard_pca_image[:, :, 0])

plt.subplot(323)
plt.imshow(img_eq[:, :, 0])

plt.subplot(324)
plt.imshow(eq_pca[:, :, 0])

plt.subplot(325)
plt.imshow(img_rescale[:, :, 0])

plt.subplot(326)
plt.imshow(stretch_pca[:, :, 0])

plt.show()

In [ ]:
img_rescale2 = img_rescale.reshape(625, 632, 3)


plt.imshow(img_rescale2[:, :, 0])

In [ ]:
ss = im[:, :, 80]
ss.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(ss)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(kk[:, :, 0])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(kks[:, :, 0])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(3)
X = im.reshape(-1, 474)

pca.fit(X)
uu = pca.transform(X)
pca.explained_variance_ratio_

In [ ]:
uu = uu.reshape(50, 50, 3)
uu.shape
plt.imshow(uu[:, :, 0])

In [ ]:
from skimage import exposure

# Equalization
img_eq = exposure.equalize_hist(im)
img_eq.shape

In [ ]:
plt.imshow(img_eq[:, :, 0])

In [ ]:
pca = PCA(30)
Xx = img_eq.reshape(-1, 474)

pca.fit(Xx)
ulu = pca.transform(Xx)
pca.explained_variance_ratio_

In [ ]:
ulu = ulu.reshape(50, 50, 30)
ulu.shape
plt.imshow(ulu[:, :, 9])

In [ ]:
# Contrast stretching
p2, p98 = np.percentile(im, (2, 98))
img_rescale = exposure.rescale_intensity(im, in_range=(p2, p98))
plt.imshow(img_rescale[:, :, 0])

In [ ]:
pca = PCA(30)
Xxx = img_rescale.reshape(-1, 474)

pca.fit(Xxx)
ulssu = pca.transform(Xxx)
pca.explained_variance_ratio_

In [ ]:
ulssu = ulssu.reshape(50, 50, 30)
ulssu.shape
plt.imshow(ulssu[:, :, 0])

In [ ]:
import numpy as np
from PIL import Image, ImageEnhance

im = Image.open("../Sorted_tiles/1.jpg")
im2 = imageio.imread("../Sorted_tiles/1.jpg")

In [ ]:
print(np.array(im).shape)
print(im2.shape)
print(im - im2)

In [ ]:
import imageio
import numpy as np

im = imageio.imread("../Sorted_tiles/stripe_merged_2_11_18.tif")
np.shape(im)

In [ ]:
im2 = im[:, :, :10]
np.shape(im2)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Make this bigger to generate a dense grid.
N = 4

# Create some random data.
volume = np.random.rand(N, N, N)
volume = im2

# Create the x, y, and z coordinate arrays.  We use
# numpy's broadcasting to do all the hard work for us.
# We could shorten this even more by using np.meshgrid.
x = np.arange(volume.shape[0])[:, None, None]
y = np.arange(volume.shape[1])[None, :, None]
z = np.arange(volume.shape[2])[None, None, :]
x, y, z = np.broadcast_arrays(x, y, z)

# Turn the volumetric data into an RGB array that's
# just grayscale.  There might be better ways to make
# ax.scatter happy.
c = np.tile(volume.ravel()[:, None], [1, 3])

# Do the plotting in a single call.
fig = plt.figure()
ax = fig.gca(projection="3d")
ax.scatter(x.ravel(), y.ravel(), z.ravel(), c=c)

## Images to dataframe

In [63]:
import imageio
import pandas as pd

image_dataframe = pd.DataFrame(data=None, columns=["Name","target"])

image_dir_path = "../Sorted_tiles/"  # Path to images

In [64]:
import pandas as pd

labelfile = pd.read_csv("../Qgis/labeled_tiles.csv")
labelfile.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,0,stripe_merged_2_01_31.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,stripe_merged_2_01_32.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,stripe_merged_2_01_33.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,stripe_merged_2_01_34.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,stripe_merged_2_01_35.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1


In [65]:
# Stratified sample of the classes to get a balanced dataset with the same number of classes to train on.
# https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
# https://stackoverflow.com/questions/44114463/stratified-sampling-in-pandas

# Maximum number of elements in each class. If the classe have less than max every sample is included.
clasnumber_of_samples = 40

labelfile_balanced = (
    labelfile.groupby(["99"], group_keys=False)
    .apply(lambda x: x.sample(min(len(x), clasnumber_of_samples)))
    .reset_index(drop=True)
)
labelfile_balanced.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,270,stripe_merged_2_12_22.tif,1,2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1,0,0,0,0,0,0,0,0,0,0,0
1,403,stripe_merged_2_15_46.tif,1,2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1,0,0,0,0,0,0,0,0,0,0,0
2,305,stripe_merged_2_13_22.tif,1,2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1,0,0,0,0,0,0,0,0,0,0,0
3,306,stripe_merged_2_13_23.tif,1,2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1,0,0,0,0,0,0,0,0,0,0,0
4,60,stripe_merged_2_05_30.tif,5,6,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,1,1,0,0,0,0,0,0,0


In [66]:
# Size of classes
labelfile_balanced_counted = labelfile_balanced.groupby("mult_bin_labels")[
    "ID"
].nunique()
labelfile_balanced_counted

mult_bin_labels
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]    40
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]    28
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    12
Name: ID, dtype: int64

In [77]:
def get_input(path):

    img = imageio.imread(path)

    return img

data_dataframe = pd.DataFrame(data=None, columns=range(1185000))


for _, sample_image in labelfile_balanced.iterrows():

    # Creates the file paths for the image
    image_sample_path = image_dir_path + sample_image["Tile_name"]

    # Reads in each image as array, performs argumentation on each image
    input_raw = get_input(image_sample_path)

    # Flatten image array
    image_flattened = input_raw.flatten()

    data = image_flattened.reshape(1,1185000)
    data_dataframe = data_dataframe.append(data)
    
    image_dataframe = image_dataframe.append(
        {
            "Name": sample_image["Tile_name"],
            "target": sample_image["99"],
        },
        ignore_index=True,
    )

KeyboardInterrupt: 

In [68]:
ss = image_flattened.reshape(1,1185000)
ss

Array([[2.6214754e-02, 2.7319100e-02, 2.8618366e-02, ..., 0.0000000e+00,
        0.0000000e+00, 2.1278711e-05]], dtype=float32)

In [ ]:
data_dataframe.head()

In [69]:
image_dataframe.head()

,Name,target
0,stripe_merged_2_12_22.tif,0
1,stripe_merged_2_15_46.tif,0
2,stripe_merged_2_13_22.tif,0
3,stripe_merged_2_13_23.tif,0
4,stripe_merged_2_05_30.tif,0


In [70]:
image_dataframe.to_csv("test.csv")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    image_dataframe["data"].values,
    image_dataframe["target"].values,
    test_size=0.8,
    random_state=42,
)

clf = RandomForestClassifier(n_estimators=10, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
sss = image_dataframe["data"].values

In [ ]:
clf.score(X_test, y_test)

In [ ]:
# Samples the file for a image, removes index so it can be used as loc[0]
    label_sample = label_file.sample().reset_index(drop=True)

    # Creates the file paths for the image
    image_sample_path = image_path + label_sample['Tile_name'].item()

    # Reads in each image as array, performs argumentation on each image
    input_raw = get_input(image_sample_path)
    input_permutated = random_image_flip_rotation(input_raw)

    # Reshape model to (n*k*3)
    #input_permutated = np.reshape(input_permutated, (632,625,3))
    from sklearn.decomposition import PCA
    pca = PCA(3)
    input_permutated = input_permutated.reshape(-1, 474)
    input_permutated = pca.fit_transform(input_permutated)
    input_permutated = input_permutated.reshape(50,50, 3)

    # Add a new dimention to the array because Keras wants it this way?
    input_new_dim = np.expand_dims(input_permutated, axis=0)


    # Set a label by using the tile name. This line first finds the rowdata for 
    # a spesific tile and gives it out as a dataframe. Then the .iloc function 
    # finds the values of the colums 1 up to 99 and give them out as a list(list in a list). 
    label = label_sample.loc[0, ['1','2','3','4','5','6','7','8','9','10','11','12','99']].values

    # Place the labels inside a list in a list because Keras wants it this way??
    label = [[label]]

    yield(input_new_dim, label)